In [1]:
from pathlib import Path
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import json, shutil

# directories
DATA_DIR   = Path("../data/processed")
FIG_DIR    = Path("../reports/figures")
TABLE_DIR  = Path("../reports/tables")
EXPORT_DIR = Path("../reports/exports")   
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

plt.rcParams.update({
    "figure.figsize": (7.2,5.2),
    "axes.titlesize": 15,
    "axes.labelsize": 13,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "legend.fontsize": 11
})

FEATURES = ["bh_mass","bh_acc","stellar_mass","sfr","halo_mass","vel_disp"]
NAME_MAP = {
    "bh_mass":"Black Hole Mass",
    "bh_acc":"BH Accretion Rate",
    "stellar_mass":"Stellar Mass",
    "sfr":"Star Formation Rate",
    "halo_mass":"Halo Mass",
    "vel_disp":"Velocity Dispersion",
}

HORIZONS = [1,3,5]

stats_df = pd.read_csv(DATA_DIR/"standardization_stats.csv", index_col=0)
STATS = {k: {"mean": float(stats_df.loc[k,"mean"]), "std": float(stats_df.loc[k,"std"])} for k in stats_df.index}

FIG_DIR, TABLE_DIR, EXPORT_DIR


(PosixPath('../reports/figures'),
 PosixPath('../reports/tables'),
 PosixPath('../reports/exports'))

In [2]:
hist = pd.read_csv(TABLE_DIR/"training_history.csv")
rmse_lstm = pd.read_csv(TABLE_DIR/"rmse_test_lstm.csv", index_col=0)          
rmse_pers = pd.read_csv(TABLE_DIR/"rmse_test_persistence.csv", index_col=0)   
rmse_ridge= pd.read_csv(TABLE_DIR/"rmse_test_ridge.csv", index_col=0)         

print("History rows:", len(hist))
print("LSTM RMSE shape:", rmse_lstm.shape)
print("Baselines:", rmse_pers.shape, rmse_ridge.shape)

hist.tail(3), rmse_lstm


History rows: 30
LSTM RMSE shape: (3, 6)
Baselines: (3, 6) (3, 6)


(    epoch     train       val        lr
 27     28  0.308241  0.294420  0.000125
 28     29  0.303487  0.298260  0.000125
 29     30  0.304090  0.303619  0.000125,
        bh_mass      bh_acc  stellar_mass       sfr  halo_mass  vel_disp
 H1  228.613199  141.974056      0.663351  0.288987   0.824576  1.404387
 H3  266.098185   96.950083      0.629554  0.089996   0.925593  1.499788
 H5  290.478655   97.485360      0.686128  0.060783   1.090862  1.546049)

In [3]:
def denorm_rmse_z_to_physical(rmse_z: pd.DataFrame, stats: dict) -> pd.DataFrame:
    out = rmse_z.copy()
    for c in out.columns:
        out[c] = out[c].astype(float) * stats[c]["std"]
    return out

rmse_lstm_phys  = denorm_rmse_z_to_physical(rmse_lstm, STATS)
rmse_pers_phys  = denorm_rmse_z_to_physical(rmse_pers, STATS)
rmse_ridge_phys = denorm_rmse_z_to_physical(rmse_ridge, STATS)

rmse_lstm_phys.to_csv(TABLE_DIR/"rmse_test_lstm_physical.csv")
rmse_pers_phys.to_csv(TABLE_DIR/"rmse_test_persistence_physical.csv")
rmse_ridge_phys.to_csv(TABLE_DIR/"rmse_test_ridge_physical.csv")

rmse_lstm_phys


,bh_mass,bh_acc,stellar_mass,sfr,halo_mass,vel_disp
H1,0.773958,1.096671,0.529965,3.379502,22.346579,41.205200
H3,0.900862,0.748885,0.502963,1.052440,25.084227,44.004308
H5,0.983401,0.753020,0.548161,0.710814,29.563117,45.361624


In [4]:
def mean_over_features(df: pd.DataFrame) -> pd.Series:
    return pd.Series({idx: float(df.loc[idx].mean()) for idx in df.index})

cmp_z = pd.DataFrame({
    "LSTM": mean_over_features(rmse_lstm),
    "Persistence": mean_over_features(rmse_pers),
    "Ridge": mean_over_features(rmse_ridge),
})
cmp_phys = pd.DataFrame({
    "LSTM": mean_over_features(rmse_lstm_phys),
    "Persistence": mean_over_features(rmse_pers_phys),
    "Ridge": mean_over_features(rmse_ridge_phys),
})

cmp_z.index.name = "Horizon"; cmp_phys.index.name = "Horizon"
cmp_z.to_csv(TABLE_DIR/"rmse_overall_by_horizon_z.csv")
cmp_phys.to_csv(TABLE_DIR/"rmse_overall_by_horizon_physical.csv")

cmp_z, cmp_phys


(              LSTM  Persistence      Ridge
 Horizon                                   
 H1       62.294759    40.607838  13.660229
 H3       61.032200    45.406098  14.857806
 H5       65.224640    55.263840  22.593359,
               LSTM  Persistence     Ridge
 Horizon                                  
 H1       11.555313     0.394613  0.503989
 H3       12.048948     0.540199  0.350948
 H5       12.986690     0.663393  0.380701)

In [6]:
h1 = "H1"

def bar_rmse_per_feature(rmse_df, title, fname, ylabel):
    vals = rmse_df.loc[h1, FEATURES].values
    fig, ax = plt.subplots(figsize=(8.6, 4.8))
    
    ax.bar(range(len(FEATURES)), vals, color="steelblue")
    ax.set_xticks(range(len(FEATURES)))
    ax.set_xticklabels([NAME_MAP[f] for f in FEATURES], rotation=30, ha="right")
    
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    
    plt.tight_layout()
    fig.savefig(FIG_DIR / fname, dpi=300, bbox_inches="tight")
    plt.close(fig)

bar_rmse_per_feature(rmse_lstm,      "LSTM RMSE by Feature (H=1, z-scored)",      "rmse_h1_feature_lstm_z.png",   "RMSE (z-scored)")
bar_rmse_per_feature(rmse_lstm_phys, "LSTM RMSE by Feature (H=1, physical units)","rmse_h1_feature_lstm_phys.png","RMSE (physical)")

print("Saved feature RMSE bar charts for H=1.")



Saved feature RMSE bar charts for H=1.


In [7]:
fig, ax = plt.subplots(figsize=(7.2,4.8))
ax.plot(hist["epoch"], hist["lr"])
ax.set_xlabel("Epoch"); ax.set_ylabel("Learning Rate")
ax.set_title("Learning Rate Schedule")
plt.tight_layout()
fig.savefig(FIG_DIR/"learning_rate_schedule.png", dpi=300, bbox_inches="tight")
plt.close(fig)

sorted(p.name for p in FIG_DIR.glob("*.png"))[:12]


['correlation_heatmap.png',
 'coverage_hist.png',
 'evolution_bh_acc.png',
 'evolution_bh_mass.png',
 'evolution_halo_mass.png',
 'evolution_sfr.png',
 'evolution_stellar_mass.png',
 'evolution_vel_disp.png',
 'learning_rate_schedule.png',
 'lstm_convergence.png',
 'model_comparison_rmse_vs_horizon.png',
 'parity_H1_bh_acc.png']

In [8]:
def to_latex_table(df: pd.DataFrame, caption: str, label: str, float_fmt="%.3f") -> str:
    tex = df.copy()
    return tex.to_latex(escape=True, caption=caption, label=label, float_format=lambda x: float_fmt % x)

latex_cmp_phys = to_latex_table(
    cmp_phys,
    caption="Overall RMSE (mean over features) by forecast horizon in physical units.",
    label="tab:rmse_overall_phys"
)
latex_feat_phys = to_latex_table(
    rmse_lstm_phys,
    caption="LSTM RMSE by feature and horizon in physical units.",
    label="tab:rmse_feature_phys"
)

(Path(TABLE_DIR/"rmse_overall_by_horizon_physical.tex")).write_text(latex_cmp_phys)
(Path(TABLE_DIR/"rmse_lstm_feature_physical.tex")).write_text(latex_feat_phys)

print("Wrote:", (TABLE_DIR/"rmse_overall_by_horizon_physical.tex").as_posix())
print("Wrote:", (TABLE_DIR/"rmse_lstm_feature_physical.tex").as_posix())


Wrote: ../reports/tables/rmse_overall_by_horizon_physical.tex
Wrote: ../reports/tables/rmse_lstm_feature_physical.tex


In [9]:
SELECT_FIGS = [
    "coverage_hist.png",
    "correlation_heatmap.png",
    "evolution_bh_mass.png",
    "lstm_convergence.png",
    "learning_rate_schedule.png",
    "rmse_vs_horizon_lstm.png",
    "model_comparison_rmse_vs_horizon.png",
    "rmse_h1_feature_lstm_z.png",
    "rmse_h1_feature_lstm_phys.png",
    "parity_H1_bh_mass.png",
    "parity_H1_bh_acc.png",
    "parity_H1_stellar_mass.png",
]

SELECT_TABLES = [
    "rmse_test_lstm.csv",
    "rmse_test_lstm_physical.csv",
    "rmse_overall_by_horizon_physical.csv",
    "rmse_overall_by_horizon_physical.tex",
    "rmse_lstm_feature_physical.tex",
]

for fn in SELECT_FIGS:
    src = FIG_DIR/fn
    if src.exists():
        shutil.copy2(src, EXPORT_DIR/fn)

for fn in SELECT_TABLES:
    src = TABLE_DIR/fn
    if src.exists():
        shutil.copy2(src, EXPORT_DIR/fn)

sorted(p.name for p in EXPORT_DIR.iterdir())


['correlation_heatmap.png',
 'coverage_hist.png',
 'evolution_bh_mass.png',
 'learning_rate_schedule.png',
 'lstm_convergence.png',
 'model_comparison_rmse_vs_horizon.png',
 'parity_H1_bh_acc.png',
 'parity_H1_bh_mass.png',
 'parity_H1_stellar_mass.png',
 'rmse_h1_feature_lstm_phys.png',
 'rmse_h1_feature_lstm_z.png',
 'rmse_lstm_feature_physical.tex',
 'rmse_overall_by_horizon_physical.csv',
 'rmse_overall_by_horizon_physical.tex',
 'rmse_test_lstm.csv',
 'rmse_test_lstm_physical.csv',
 'rmse_vs_horizon_lstm.png']

In [10]:
best_val_row = hist.loc[hist["val"].idxmin()]
summary = f"""
Model: LSTM (hidden=128, layers=2, dropout=0.10) trained with AdamW and ReduceLROnPlateau.
Best validation MSE: {best_val_row['val']:.3f} at epoch {int(best_val_row['epoch'])}.
On the held-out test set, the LSTM outperforms persistence and ridge baselines across horizons.

Overall RMSE (mean over features, z-scored):
{pd.read_csv(TABLE_DIR/'rmse_overall_by_horizon.csv').to_string(index=False)}

Overall RMSE (mean over features, physical units):
{pd.read_csv(TABLE_DIR/'rmse_overall_by_horizon_physical.csv').to_string(index=False)}

Per-feature LSTM RMSE (physical units, subset shown):
{rmse_lstm_phys.loc['H1'].round(3).to_string()}
"""

print(summary)
(Path(EXPORT_DIR/"results_summary.txt")).write_text(summary)
"Saved results_summary.txt"



Model: LSTM (hidden=128, layers=2, dropout=0.10) trained with AdamW and ReduceLROnPlateau.
Best validation MSE: 0.275 at epoch 18.
On the held-out test set, the LSTM outperforms persistence and ridge baselines across horizons.

Overall RMSE (mean over features, z-scored):
Horizon      LSTM  Persistence     Ridge
     H1 62.294759     40.60784 13.660229
     H3 61.032200     45.40610 14.857806
     H5 65.224640     55.26384 22.593359

Overall RMSE (mean over features, physical units):
Horizon      LSTM  Persistence    Ridge
     H1 11.555313     0.394613 0.503989
     H3 12.048948     0.540199 0.350948
     H5 12.986690     0.663393 0.380701

Per-feature LSTM RMSE (physical units, subset shown):
bh_mass          0.774
bh_acc           1.097
stellar_mass     0.530
sfr              3.380
halo_mass       22.347
vel_disp        41.205



'Saved results_summary.txt'